In [3]:
import pandas as pd
import re
import requests
import os

from dotenv import load_dotenv

#.env 파일 로드
load_dotenv()
kakao_api_key = os.getenv("kakao_api_key")


### 청약매물 목록 위도, 경도, 행정구역 컬럼 추가

In [6]:
def pre_col_gen(df):

    # 중복된 '공고번호' 제거 (첫 번째 항목만 남김)
    df_copy = df.reset_index(drop=True).drop_duplicates(subset="공고번호", keep='first', ignore_index=True)
    
    # '정제_공급위치' 열 생성 후 간단한 전처리
    df_copy["정제_공급위치"] = df_copy["공급위치"].apply(lambda address: re.sub(r'\([^)]*\)', '', address).replace('블록', '').replace('일원', '').replace('공동', '').replace('일반', '').replace('  ', ' ').strip())
    
    # 필요한 열만 선택
    df_copy = df_copy[["공고번호", "정제_공급위치"]]
    
    return df_copy

def process_address_data(df):
    # 카카오맵 API 사용해서 위도 경도 추출
    kakao_api_url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {kakao_api_key}"}

    def get_lat_lon_kakao(address):
        try:
            response = requests.get(kakao_api_url, headers=headers, params={"query": address})
            if response.status_code == 200:
                result = response.json()
                if result['documents']:
                    lat = result['documents'][0]['y']
                    lon = result['documents'][0]['x']
                    return lat, lon
            return None, None
        except Exception as e:
            print(f"Error fetching data for address {address}: {e}")
            return None, None

    df['위도'], df['경도'] = zip(*df['정제_공급위치'].apply(lambda x: get_lat_lon_kakao(x)))

    return df

def process_address(df):
    
    def split_and_update_address(address):
        parts = address.split(' ')
        
        if len(parts) >= 3:
            sido = parts[0]
            sigungu = parts[1]
            eupmyeondong = parts[2]
            updated_address = f"{sido} {sigungu} {eupmyeondong}"
            return sido, sigungu, eupmyeondong, updated_address
        else:
            return None, None, None, address
    
    # 주소 분리 및 업데이트
    df[['시도', '시군구', '읍면동', '정제_공급위치_업데이트']] = df['정제_공급위치'].apply(lambda x: pd.Series(split_and_update_address(x)))
    
    # 기존 정제_공급위치 열을 업데이트함
    df['정제_공급위치'] = df['정제_공급위치_업데이트']
    
    # 불필요한 열 제거
    df = df.drop(columns=['정제_공급위치_업데이트'])
    
    return df


def lat_lon_pipeline(df):

    # 서울, 경기, 인천 지역만 필터링
    df = df[df['공급지역명'].isin(['서울', '경기', '인천'])]

    df_copy = pre_col_gen(df)

    # Kakao API로 위도와 경도 가져오기
    df_copy = process_address_data(df_copy)
    
    # 주소 분리 및 업데이트 
    df_copy = process_address(df_copy)

    # 위도와 경도가 None인 행만 선택하여 다시 처리
    df_none = df_copy[(df_copy['위도'].isnull()) & (df_copy['경도'].isnull())].copy()
    
    if not df_none.empty:
        df_none_processed = process_address_data(df_none)
        df_copy.loc[df_none.index, ['위도', '경도']] = df_none_processed[['위도', '경도']]
    
    df_copy.drop("정제_공급위치", axis=1, inplace=True)
    
    return df, df_copy


In [7]:
df = pd.read_csv("C:/Users/user/mid_project/src/storage/raw_data/병합_청약매물_목록_정보.csv", encoding='cp949')
df, df_copy = lat_lon_pipeline(df)
df_merged = pd.merge(df, df_copy, on='공고번호', how='left')

In [9]:
df_merged.to_csv('병합_청약매물_목록_정보 (서울, 경기, 인천).csv', index=False, encoding='cp949')

In [12]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19630 entries, 0 to 19629
Data columns (total 71 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   주택관리번호        19630 non-null  int64  
 1   공고번호          19630 non-null  int64  
 2   주택명           19630 non-null  object 
 3   주택구분코드        19630 non-null  int64  
 4   주택구분코드명       19630 non-null  object 
 5   주택상세구분코드      19630 non-null  int64  
 6   주택상세구분코드명     19630 non-null  object 
 7   분양구분코드        19630 non-null  int64  
 8   분양구분코드명       19630 non-null  object 
 9   공급지역코드        19630 non-null  int64  
 10  공급지역명         19630 non-null  object 
 11  공급위치우편번호      19630 non-null  int64  
 12  공급위치          19630 non-null  object 
 13  공급규모          19630 non-null  int64  
 14  모집공고일         19630 non-null  object 
 15  청약접수시작일       19630 non-null  object 
 16  청약접수종료일       19630 non-null  object 
 17  특별공급접수시작일     18076 non-null  object 
 18  특별공급접수종료일     18076 non-nu